In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from tqdm import tqdm
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Mount Drive to Access Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**IMPORTING NECESSARY LIBRARIES AND T5 MODEL**

In [ ]:
!pip install happytransformer
import re
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import STOPWORDS, WordCloud
from happytransformer import HappyTextToText, TTSettings, TTTrainArgs
from transformers import T5Tokenizer

**CONVERSION OF M2 FORMAT TO TXT**

In [ ]:
!tar xvzf '/content/drive/MyDrive/lang8.bea19.tar.gz'

lang8.train.auto.bea19.m2
lang8_to_m2.py
readme.txt


In [ ]:
m2 = open("/content/lang8.train.auto.bea19.m2").read().strip().split("\n\n")
out = open("fce.train.gold.bea19.txt", "w")

# Do not apply edits with these error types
skip = {"noop", "UNK", "Um"}

for sent in m2:
    sent = sent.split("\n")
    cor_sent = sent[0].split()[1:]  # Ignore "S "
    edits = sent[1:]
    offset = 0
    for edit in edits:
        edit = edit.split("|||")
        if len(edit) < 2:
            continue  # Skip edits without at least 2 elements
        if edit[1] in skip:
            continue
        coder = int(edit[-1])
        if coder != 0:
            continue  # Ignore other coders
        span = edit[0].split()[1:]  # Ignore "A "
        start = int(span[0])
        end = int(span[1])
        cor = edit[2].split()
        cor_sent[start + offset:end + offset] = cor
        offset = offset - (end - start) + len(cor)
    out.write(" ".join(cor_sent) + "\n")


**PREPROCESSING INCORRECT SENTENCE**

In [ ]:
file1 = open("/content/lang8.train.auto.bea19.m2","r")
s1 = file1.read()

each_sent = s1.split("\n\n")

incorrect = []
for i in range(len(each_sent)):
    temp = each_sent[i].split("\n")
    temp = temp[0]
    temp = temp.split(" ")
    temp = temp[1:]# ignore S
    temp = ' '.join(temp)
    incorrect.append(temp)

**PREOROCESSING CORRECT SENTENCE**

In [ ]:
file2 = open("fce.train.gold.bea19.txt","r")
s2 = file2.read()

correct = s2.split("\n")


**STORING CORRECT AND INCORRECT SENTENCE IN DATAFRAME**

In [ ]:
df = pd.DataFrame()
df["target"] = correct
df["input"] = incorrect

**STORE IN CSV FORMAT**

In [ ]:
df.to_csv("data.csv",index=False)

index = []
for i in range(len(df.values)):
    if df.values[i][0] == df.values[i][1]:
        index.append(i)

df = df.drop(index)
df.shape



(498360, 2)

**CHECK NULL VALUES**

In [ ]:
df.isnull().values.any()

df.dropna(inplace=True)

**CHECK DUPLICATE VALUE**

In [ ]:
df.duplicated().values.any()

df.drop_duplicates(inplace=True)
df.reset_index(inplace=True,drop=True)

**TEXT CLEANING**

In [ ]:
def clean(text):
   # removes characters inside (),{},[] and <>
   # removes characters like -+@#^/|*(){}$~`
   # we not not removing ,.!-:;"'

    text = re.sub('<.*>', '', text)
    text = re.sub('\(.*\)', '', text)
    text = re.sub('\[.*\]', '', text)
    text = re.sub('{.*}', '', text)
    text = re.sub("[-+@#^/|*(){}$~`<>=_]","",text)
    text = text.replace("\\","")
    text = re.sub("\[","",text)
    text = re.sub("\]","",text)
    text = re.sub("[0-9]","",text)
    return text



**LOWERCASE CONVERSION**

In [ ]:
def word_cloud(df,col):

    comment_words = ''
    stopwords = set(STOPWORDS)

# iterate through the csv file
    for val in df[col]:
        val = str(val)
    # split the value
        tokens = val.split()
    # Converts each token into lowercase
        for i in range(len(tokens)):
            tokens[i] = tokens[i].lower()

        comment_words += " ".join(tokens)+" "


**SPLIT DATA INTO TRAIN AND EVALUATION**

In [ ]:
train_ratio = 0.7
train_size = int(train_ratio * len(df))
train_data = df[:train_size]
eval_data = df[train_size:]

train_data.to_csv("train.csv", index=False)
eval_data.to_csv("eval.csv", index=False)

**TRUNCATE AND LOAD TRAIN DATASET**

In [ ]:
df = pd.read_csv("train.csv")

desired_length = 250000
truncated_df = df[:desired_length]

truncated_df.to_csv("t_train.csv", index=False)

**TOKENIZATION**

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

input_sentences = train_data["input"].tolist()
input_encodings = tokenizer(input_sentences, padding=True, truncation=True, return_tensors="pt")

target_sentences = train_data["target"].tolist()
target_encodings = tokenizer(target_sentences, padding=True, truncation=True, return_tensors="pt")

train_data["input_ids"] = input_encodings.input_ids.tolist()
train_data["attention_mask"] = input_encodings.attention_mask.tolist()
train_data["target_ids"] = target_encodings.input_ids.tolist()
train_data["target_attention_mask"] = target_encodings.attention_mask.tolist()


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:238: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

**MODEL TRAINING**

In [ ]:
happy_tt = HappyTextToText("T5", "t5-base")


args = TTTrainArgs(batch_size=8)
happy_tt.train("train.csv", args=args)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/312693 [00:00<?, ? examples/s]

Map:   0%|          | 0/34744 [00:00<?, ? examples/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
1,1.450400,1.386198
3909,0.666900,0.571037
7818,0.613500,0.555068
11727,0.596200,0.544693
15636,0.583200,0.535152
19545,0.577900,0.530261
23454,0.574400,0.530726


Step,Training Loss,Validation Loss
1,1.450400,1.386198
3909,0.666900,0.571037
7818,0.613500,0.555068
11727,0.596200,0.544693
15636,0.583200,0.535152
19545,0.577900,0.530261
23454,0.574400,0.530726
27363,0.568100,0.523544
31272,0.573500,0.522119
35181,0.566500,0.521160


**BEAM SEARCH ALGORITHM**

In [ ]:
def generate_text(input_text, happy_tt, tokenizer, beam_settings):
    input_text = "grammar: " + input_text
    result = happy_tt.generate_text(input_text, args=beam_settings)
    return result.text

beam_settings =  TTSettings(num_beams=2, min_length=1, max_length=50)

**EXAMPLES**

In [ ]:
example_1 = "I is girl"
result_1 = generate_text(example_1, happy_tt, tokenizer, beam_settings)
print(result_1)

grammar: I am a girl .


In [ ]:
example_2 = "I am enjoys, writtings articles ons AI and I also enjoyed write articling on AI."
result_2 = generate_text(example_2, happy_tt, tokenizer, beam_settings)
print(result_2)


grammar: I enjoy writing articles on AI and I also enjoy writing articling on AI .


In [ ]:
import pickle

In [ ]:
pickle.dump(happy_tt, open('/content/drive/MyDrive/model','wb'))

In [ ]:
!pip install joblib

In [ ]:
import joblib

# Assuming happy_tt is the object you want to save

# To save the object using joblib
joblib.dump(happy_tt, '/content/drive/MyDrive/model.joblib')

# To load the object from the saved file
loaded_happy_tt = joblib.load('/content/drive/MyDrive/model.joblib')


In [ ]:
!pip install nltk
!pip install happytransformer
!pip install joblib
import nltk
from happytransformer import HappyTextToText
l_happy_tt = joblib.load('/content/drive/MyDrive/model.joblib')

# Define reference and hypothesis lists
references = ["Jack is a boy. He cooks good food for us.", "I love your dress. It is so pretty."]
hypotheses = []

# Generate hypotheses using your model for input sentences
input_sentences = ["Jack a boy. She cook food for we", "I love you dress. It at very pretty"]

for sentence in input_sentences:
    # Generate a hypothesis for each input sentence
    hypothesis = generate_text(sentence, loaded_happy_tt, tokenizer, beam_settings)
    hypotheses.append(hypothesis)

# Calculate GLEU score
gleu_score = nltk.translate.gleu_score.corpus_gleu(references, hypotheses)

print(f"GLEU Score: {gleu_score}")

GLEU Score: 0.005681818181818182


In [ ]:
#Load the saved model that is trained on the dataset
loaded_happy_tt = joblib.load('/content/drive/MyDrive/model.joblib')


In [ ]:
example_1 = "" #Your Input Here
result_1 = generate_text(example_1, loaded_happy_tt, tokenizer, beam_settings)
print(result_1)